# **Advanced Feature Engineering in BQML**

**Learning Objectives**

1. Evaluate the model
2. Extract temporal features, feature cross temporal features
3. Apply `ML.FEATURE_CROSS` to categorical features
4. Create a Euclidian feature column, feature cross coordinate features
5. Apply the `BUCKETIZE` function, `TRANSFORM` clause, L2 Regularisation

## **Introduction**

In this lab, we utilise feature engineering to improve the prediction of the fare amount for a taxi ride in New York City. We will use BigQuery ML to build a taxifare prediction model, using feature engineering to improve and create a final model.

In this notebook, we perform a feature cross using BigQuery's `ML.FEATURE_CROSS`, derive coordinate features, feature cross coordinate features, clean up the code, apply the `BUCKETIZE` function, the `TRANSFORM` clause, L2 Regularisation, and evaluate model performance throughout the process.

## **Set up environment variables and load necessary libraries**

In [ ]:
# Install Google Cloud BigQuery
!pip install --user google-cloud-bigquery=1.25.0

In [ ]:
%%bash

export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is "$PROJECT

## **The source data set**

Our data set is hosted in BigQuery. The taxi fare data is a publically available data set, meaning anyone with a GCP account has access.

The Taxi Fare data set is relatively large at 55 million training rows, but simple to understand, with only 6 features. The `fare_amount` is the target, the continuous value we'll train a model to predict.

## **Create a BigQuery data set**

A BigQuery data set is a container for tables, views, and models built with BigQuery ML. Let's create one called **feat_eng**. Weel do the same for a GCS bucket for our project too.

In [ ]:
%%bash

# Create a BigQuery data set for feat_eng if it doesn't exist
datasetexists=$(bq ls -d | grep -w feat_eng)

if [ -n "$datasetexists"]; then
    echo -e "BigQuery dataset already exists, let's not recreate it"

else
    echo "Creating BigQuery dataset titled: feat_eng"
    
    bq --location=US mk --dataset \
        --description "Taxi Fare" \
        $PROJECT:feat_eng
    echo "\nHere are your current datasets:"
    bq ls
fi

## **Creating the training table**

Since there is already a publicly available data set, we can simply create the training data table using this raw input data. Note the `WHERE` clause in the below query: This clause allows us to TRAIN a portion of the data (e.g. one hundred thousand rows versus one million rows), which keeps our query costs down.

In [ ]:
%%bigquery

# Creating the table in our data set
CREATE OR REPLACE TABLE
    feat_eng.feateng_training_data AS
SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    passenger_count * 1.0 AS passengers,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat
FROM
    `nyc-tlc.yellow.trips`
WHERE
    MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 10000) = 1
    AND fare_amount >= 2.5
    AND passenger_count > 0
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45

## **Verify table creation**

Verify that you created the data set

In [ ]:
%%bigquery

# LIMIT 0 is a free query; this allows us to check that the table exists
SELECT
    *
FROM
    feat_eng.feateng_training_data
LIMIT
    0

## **Baseline Model: Create the baseline model**